<table>
 <tr align=left><td><img align=left src="./images/CC-BY.png">
 <td>Text provided under a Creative Commons Attribution license, CC-BY. All code is made available under the FSF-approved MIT license. (c) Kyle T. Mandli</td>
</table>

In [4]:
from __future__ import print_function

%matplotlib inline
import numpy
import matplotlib.pyplot as plt
import matplotlib.animation
from IPython.display import HTML
from clawpack import pyclaw
from clawpack import riemann

# Nonlinear Scalar Conservation Laws

Up until now we have mostly assumed that our PDE
$$
    q_t + f(q)_x = 0
$$
has been linear, i.e. $f(q) = A q$.  Part of this was to establish the bsaic theory while we know what the true solution should look like.  We now will investigate how nonlinear problems do not simply translate but can deform the solution in interesting ways.  Note that we will assume that $f(q)$ is convex/concave, i.e. $f''(q)$ does not change sign.

As a test problem we will consider basic traffic flow as our PDE of choice.  For this we assume that we can model traffic, or the density of cars, as a continuum as we do for fluids.  This leads us to the conservation law
$$
    q_t + (U(q) q)_x = 0
$$
where $q$ represents the density of cars and $U(q)$ the speed of those cars.  We then need to specify the relationship between the speed at which the cars travel and the local density of cars.

The Lighthill, Whitham, and Richards (LWR) equation uses a simple function for the speed that is
$$
    U(q) = u_{\text{max}}(1 - q) \text{   for } 0 \leq q \leq 1
$$
where $u_{\text{max}}$ is the speed limit and assumed constant.  Adding this to our previus equation leads us to a flux defined as
$$
    f(q) = u_{\text{max}}(1 - q) q
$$
that is concave with respect to $q$.

In [5]:
def traffic_animation():
    solver = pyclaw.ClawSolver1D(riemann.traffic_1D)
    solver.bc_lower[0] = pyclaw.BC.extrap
    solver.bc_upper[0] = pyclaw.BC.extrap

    x = pyclaw.Dimension(-1.0,1.0,500,name='x')
    domain = pyclaw.Domain(x)
    num_eqn = 1
    state = pyclaw.State(domain,num_eqn)

    grid = state.grid
    xc=grid.p_centers[0]

    
    state.q[0, :] = 0.75 * (xc < 0)
    state.q[0, :] = 0.75 * (xc > 0)
#     state.q[0, :] = 1.0 * numpy.exp(-xc**2)

    state.problem_data['efix']=True
    state.problem_data['umax']=1.

    claw = pyclaw.Controller()
    claw.tfinal =2.0
    claw.solution = pyclaw.Solution(state,domain)
    claw.solver = solver
    claw.keep_copy = True
    
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((-0.1, 1.1))
    axes.set_title("Traffic Flow")
    
    def init():
        axes.set_xlim((x[0], x[-1]))
        axes.set_ylim((-0.1, 1.1))
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'bo-')
        return (computed_line, )
    
    computed_line, = init()
    
    def fplot(n):
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        return (computed_line, )

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)
HTML(traffic_animation().to_jshtml())

Let us first examine how the characteristics behave in this simple case where the initial condition is smooth but tleads to a dicontinuous solution.  If we think of the $k$th car whose trajectory is $X_k(t)$.  If we assume that the $k$th car makes a decision on how fast she goes $U_(q_k(t))$ then we have
$$
    X'_k(t) = U_k(q_k(t)) = U([X_{k+1}(t) - X_k(t)]^{-1}) \quad \text{for} ~ k=1,\ldots,m
$$
with the boundaries appropriately specified.

Note that for these test cases that the characteristic speed agrees with the speed at which the particles are moving.  This is a general property of nonlinear hyperbolic PDEs and arises as $f'(q) \neq u$ as it is in the linear case.

## Quasilinear Forms and Characteristics

One useful way we can analyze what may go on given a nonlinear flux is to study the quasilinear form of the PDE defined as
$$
    q_t + f'(q)q_x = 0.
$$
If $q(x,t)$ is smooth then we would have
$$
    X'(t) = f'(q(X(t), t))
$$
and
$$
    \frac{\text{d}}{\text{d}t} q(X(t), t) = X'(t) q_x + q_t = 0
$$
and therefore $q(x,t)$ is constant on the characteristic curves defined by $X(t)$.

As we know once we have the characteristics and know that $q$ needs to be constant on them that we need to simply trace back the characteristics to the initial condition (or boundary condition) to find out what the solution is.  Often times we represent this as
$$
    q(x,t) = q_0(\xi)
$$
where
$$
    x = \xi + f'(q_0(\xi)) t.
$$

## Burgers Equation

Another of the often used basic equations that are solved when considering nonlinear conservation laws is Burgers equation, defined as
$$
    u_t + \left( \frac{1}{2} u^2 \right)_x = 0.
$$
Sometimes this is also refered to as the inviscid Burgers equation as the Burgers equation is meant to be a simplifie dmodel of fluid flow which usually also includes viscosity in
$$
    u_t + \left( \frac{1}{2} u^2 \right)_x = \epsilon u_{xx}.
$$
We will use this eventually to define the set of solutions that are "physical".

The Burgers equation also can be written in non-conservative form as
$$
    u_t + u u_x = 0
$$
but as we will see for discontinuous solutions that we will come to different answers than the conservative form.  We will discuss this later again in relation to entropy conditions.

In [ ]:
import os
import pyclaw
path = os.path.join(os.environ.get("CLAW", os.getcwd()), "pyclaw", "fvmbook", "chap11", "burgers")
os.chdir(path)
import burgers1D

def burgers_animation():
   
    # compute the solution with the method define above:
    claw = burgers1D.burgers()
    claw.keep_copy = True
    claw.run()
    x = claw.frames[0].grid.dimensions[0].centers
    
    fig = plt.figure()
    axes = plt.subplot(1, 1, 1)
    axes.set_xlim((x[0], x[-1]))
    axes.set_ylim((-3.5, 6))
    axes.set_title("Burgers Equation")
    
    def init():
        axes.set_xlim((x[0], x[-1]))
        axes.set_ylim((-3.5, 6))
        computed_line, = axes.plot(x[0], claw.frames[0].q[0, :][0], 'bo-')
        return (computed_line, )
    
    computed_line, = init()
    
    def fplot(n):
        computed_line.set_data([x,], [claw.frames[n].q[0, :]])
        return (computed_line, )

    frames_to_plot = range(0, len(claw.frames))
    plt.close(fig)
    return matplotlib.animation.FuncAnimation(fig, fplot, frames=frames_to_plot, interval=100,
                                   blit=True, init_func=init, repeat=False)
HTML(burgers_animation().to_jshtml())

## Rarefaction Waves

## Compression Waves

## Vanishin Viscosity Limit

## Equal-Area Rule

## Shock Speed

## Rankine-Hugoniot Conditions for Systems

## Similarity Solutions and Centered Rarefactions

## Weak Solutions

## Why Should You Be Careful When Manipulating Conservation Laws?

## Nonuniqueness, Admissability, and Entropy Conditions

### Entropy Functions

### The Kruzkov Entropies

### Long-Term Behavior and N-Wave Decay